In [1]:
import sys

sys.path.append("lib/")

import os
import glob
import json
import jsonpickle

import TFilter as f
import TEventFilter as ef
from TDataType import TDataType as tdt
from TWeatherType import TWeatherType as twt
from TWeekDay import TWeekDay as twd
from TMap import TMap
from TPlotter import TPlotter
from TReader import TReader

In [2]:
PATH = "../../../DP/filtered/"

# load data
filepaths = glob.glob(os.path.join(PATH, "*.csv"))
zones_file = "data/zones.json"

LIMIT = 0  # number or 0 for no limit

reader = TReader(filepaths)

# Load TDFs
ltdfs = reader.read(LIMIT)

print(f"Loaded:\t{len(ltdfs)}")

Loaded:	1434


In [17]:
# settings
T_Line = ("*", "*", "*")
T_Date = ("01.10.2022", "31.10.2022")
T_Time = ("00:00:00", "23:59:59")
T_Vehicle = "*"

T_Data_type = tdt.BELL
T_Weather_type = twt.ALL
T_Week_day = twd.ALL

T_Map_events = True
T_Plot_events = False

T_Events_filter = True
T_Events_stop = True
T_Events_zero = True
T_Events_diff = 20

# depos zones
with open(zones_file, "r") as file:
    filter_zones = json.load(file)

# filter all ltdfs and data
data = f.filter_tdfs(ltdfs, T_Line, T_Date, T_Week_day, T_Vehicle)
print(f"TDFs:\t\t{len(data)}")

for d in data:
    d.filter_rows(T_Data_type)
    d.filter_zones(filter_zones)

# detect and filter events
events = ef.detect_events(data)
print(f"All events:\t{len(events)}")

events = ef.filter_time(events, T_Time)
print(f"In time:\t{len(events)}")

if T_Weather_type != T_Weather_type.ALL:
    events = ef.filter_weather(events, T_Weather_type)
print(f"In weather:\t{len(events)}")

if T_Events_filter:
    if T_Events_diff:
        events = ef.filter_diff(events, T_Events_diff)
    if T_Events_zero:
        events = ef.filter_zero(events)
    if T_Events_stop:
        events = ef.filter_embs(events)
    print(f"Filtered:\t{len(events)}")

TDFs:		1413
All events:	104354
In time:	104354
In weather:	104354
Filtered:	49


In [18]:
CENTER_MAP = False

m = TMap([48.1700, 17.1077], 13)

m.add_filter_zones(filter_zones)

# plot into map
if T_Map_events:
    m.add_events(events)
else:
    m.add_data(data)

m.save(CENTER_MAP)
m.save_png()

print(m.n)
m.m

49


In [5]:
p = TPlotter(events)

if T_Plot_events:
    p.plot()
    p.save()

In [19]:
# write event csv for external visualization
import pandas

data_dicts = []
for obj in events:
    event_dict = obj.__dict__.copy()
    if obj.points:  # Ensure there are points available
        event_dict["lat"] = obj.points[0][0]  # First entry latitude
        event_dict["lon"] = obj.points[0][1]  # First entry longitude
    else:
        event_dict["lat"] = None
        event_dict["lon"] = None
    del event_dict["points"]  # Remove the points list

    data_dicts.append(event_dict)  # Append modified dictionary to the list

# Convert to DataFrame and save
data_df = pandas.DataFrame(data_dicts)
data_df.to_csv("data/events.csv", index=False)

In [ ]:
# Serialize event objects
json_data = jsonpickle.encode(events, unpicklable=False, indent=4)

# Write JSON data to a file
with open("data/output.json", "w") as json_file:
    json_file.write(json_data)